<a href="https://colab.research.google.com/github/MitchellKrops/PythonAutomation/blob/main/Level_Product_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Importeerassistent voor maatschappij producten
## Importeer XML lib
import xml.etree.ElementTree as ET
from tabulate import tabulate

try:
  Bron_Bestand = input("Geef bronbestand in:")
  tree = ET.parse(Bron_Bestand)
  root = tree.getroot()

  ## Uitlezen brongegevens
  for info in root.iter('INFO'):
      waarde = info.text

  infotext = waarde.split("-")
  Bron_Maatschappij = infotext[1]
  Soort_Product = infotext[2]
  Productcode = infotext[3]
  
  EXTBRA_list = []
  for Naam_EXTBRA in root.iter('NM_EXTBRA'):
      EXTBRA_list.append(Naam_EXTBRA.text)
  
  ## Tonen brongegevens
  print()
  print("Het bronproduct bevat de volgende gegevens:")
  print()
  info_data = [[Bron_Maatschappij, Soort_Product, Productcode]]
  print (tabulate(info_data, headers=["Maatschappij", "Soort product", "Productcode"]))
  print()

  Branchecodes_list = []
  print("Het bronproduct bevat de volgende branchecodes / dekkingscodes:")

  for branchecode in root.iter('KD_BR_ELM'):
      Branchecodes_list.append(branchecode.text)
  print(Branchecodes_list)
  print()

  ## Input doelgegevens
  print()
  while True:
    Doel_Maatschappij = input("Geef doel maatschappij in:")
    if not len(Doel_Maatschappij) <= 5:
      print("Maatschappijnaam mag 5 karakters lang zijn")
      continue
    else:

      break  

  while True:  
    Doel_Productcode: str = input("Geef doel productcode in:")
    if len(Doel_Productcode) > 3 or Doel_Productcode.isdecimal() == False:
      print("Productcode mag 3 cijfers lang zijn")
      continue
    else:

      break 
    print()

  ## Aanmaken nieuw INFO Attribute
  Info_Nieuw = "-{0}-{1}-{2}".format(Doel_Maatschappij, Soort_Product, Doel_Productcode)

  ## Wegschrijven nieuw INFO Attribute
  for element in root.iter('INFO'):
      element.text = Info_Nieuw

  for productcode in root.iter('KD_PROD'):
      productcode.text = productcode.text.replace(Productcode, Doel_Productcode)

  for element in root.iter():
      try:
          element.text = element.text.replace(Bron_Maatschappij, Doel_Maatschappij)
      except AttributeError:
          pass

  ## Vastleggen en tonen branchecodes / dekkingscodes

  Branchecodes_keuze = input("Wil je branchecodes aanpassen? y/n:")
  Branchecodes_list_nieuw = []

  if Branchecodes_keuze == "y":
      for i in Branchecodes_list:
          print("Oude branchecode = " + i)
          while True:
            branchecode_nieuw = input("Nieuwe branchecode =")
            if len(branchecode_nieuw) > 3 or branchecode_nieuw.isdecimal() == False:
              print("Branchecode mag 3 cijfers lang zijn")
              continue
            else:

              break
          Branchecodes_list_nieuw.append(branchecode_nieuw)

      print("De nieuwe branchecodes zijn:")
      print(Branchecodes_list_nieuw)

      ## Vervangen van brongegevens voor doelgegevens
      Branchecodes_rubrieken = ['KD_BR_TOT', 'KD_BR_VAN', 'KD_BR_ELM']
      for i in range(len(Branchecodes_rubrieken)):
          for element in root.iter(Branchecodes_rubrieken[i]):
              index = Branchecodes_list.index(element.text)
              element.text = Branchecodes_list_nieuw[index]
              

  ## Informatie wegschrijven in Export.xml
  tree.write("{0}-{1}-{2}.xml".format(Doel_Maatschappij, Soort_Product, Doel_Productcode), method='xml', encoding='utf-8', xml_declaration=True)
  print("Succesvol importbestand aangemaakt: " + Doel_Maatschappij + "-" + Soort_Product + "-" + Doel_Productcode + ".xml")

except KeyboardInterrupt:
  print("Stop")
